In [2]:
import pandas as pd
import numpy as np
import seaborn as sns 
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings('ignore')

from collections import Counter

from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import r2_score, accuracy_score

In [ ]:
Amazon = pd.read_csv("Reviews.csv")
Amazon.head()

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
0,1,B001E4KFG0,A3SGXH7AUHU8GW,delmartian,1,1,5,1303862400,Good Quality Dog Food,I have bought several of the Vitality canned d...
1,2,B00813GRG4,A1D87F6ZCVE5NK,dll pa,0,0,1,1346976000,Not as Advertised,Product arrived labeled as Jumbo Salted Peanut...
2,3,B000LQOCH0,ABXLMWJIXXAIN,"Natalia Corres ""Natalia Corres""",1,1,4,1219017600,"""Delight"" says it all",This is a confection that has been around a fe...
3,4,B000UA0QIQ,A395BORC6FGVXV,Karl,3,3,2,1307923200,Cough Medicine,If you are looking for the secret ingredient i...
4,5,B006K2ZZ7K,A1UQRSCLF8GW1T,"Michael D. Bigham ""M. Wassir""",0,0,5,1350777600,Great taffy,Great taffy at a great price. There was a wid...


In [ ]:
Amazon.drop(['Id','Time','UserId'],axis=1,inplace=True)
Amazon.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 568454 entries, 0 to 568453
Data columns (total 7 columns):
 #   Column                  Non-Null Count   Dtype 
---  ------                  --------------   ----- 
 0   ProductId               568454 non-null  object
 1   ProfileName             568438 non-null  object
 2   HelpfulnessNumerator    568454 non-null  int64 
 3   HelpfulnessDenominator  568454 non-null  int64 
 4   Score                   568454 non-null  int64 
 5   Summary                 568427 non-null  object
 6   Text                    568454 non-null  object
dtypes: int64(3), object(4)
memory usage: 30.4+ MB


In [ ]:
Amazon.dropna(how='any',inplace=True)
Amazon.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 568411 entries, 0 to 568453
Data columns (total 7 columns):
 #   Column                  Non-Null Count   Dtype 
---  ------                  --------------   ----- 
 0   ProductId               568411 non-null  object
 1   ProfileName             568411 non-null  object
 2   HelpfulnessNumerator    568411 non-null  int64 
 3   HelpfulnessDenominator  568411 non-null  int64 
 4   Score                   568411 non-null  int64 
 5   Summary                 568411 non-null  object
 6   Text                    568411 non-null  object
dtypes: int64(3), object(4)
memory usage: 34.7+ MB


In [ ]:
Amazon['Helpful'] = Amazon['HelpfulnessNumerator']
Amazon['Not Helpful']= Amazon['HelpfulnessDenominator'] - Amazon['HelpfulnessNumerator']
Amazon.drop(['HelpfulnessNumerator','HelpfulnessDenominator'],axis=1,inplace=True)
Amazon['Helpful'].describe()

count    568411.000000
mean          1.743874
std           7.636781
min           0.000000
25%           0.000000
50%           0.000000
75%           2.000000
max         866.000000
Name: Helpful, dtype: float64

In [ ]:
Amazon['Helpful'].value_counts()

0      303813
1      116439
2       53726
3       29007
4       16963
        ...  
446         1
155         1
580         1
355         1
808         1
Name: Helpful, Length: 231, dtype: int64

In [ ]:
Amazon['Not Helpful'].value_counts()

 0      453345
 1       67248
 2       21313
 3        9144
 4        5096
         ...  
 89          1
-2           1
 52          1
 80          1
 122         1
Name: Not Helpful, Length: 89, dtype: int64

In [ ]:
Amazon['Text Length'] = Amazon['Text'].apply(lambda x: len(x.split(' ')))
Amazon['Text Length'] .describe()

count    568411.000000
mean         82.008950
std          80.808843
min           3.000000
25%          34.000000
50%          58.000000
75%         100.000000
max        3526.000000
Name: Text Length, dtype: float64

In [ ]:
Amazon['Summary Length'] = Amazon['Summary'].apply(lambda x: len(x.split(' ')))
Amazon['Summary Length'] .describe()

count    568411.000000
mean          4.128472
std           2.619384
min           1.000000
25%           2.000000
50%           4.000000
75%           5.000000
max          42.000000
Name: Summary Length, dtype: float64

In [ ]:
Amazon.head()

,ProductId,ProfileName,Score,Summary,Text,Helpful,Not Helpful,Text Length,Summary Length
0,B001E4KFG0,delmartian,5,Good Quality Dog Food,I have bought several of the Vitality canned d...,1,0,49,4
1,B00813GRG4,dll pa,1,Not as Advertised,Product arrived labeled as Jumbo Salted Peanut...,0,0,31,3
2,B000LQOCH0,"Natalia Corres ""Natalia Corres""",4,"""Delight"" says it all",This is a confection that has been around a fe...,1,0,99,4
3,B000UA0QIQ,Karl,2,Cough Medicine,If you are looking for the secret ingredient i...,3,0,43,2
4,B006K2ZZ7K,"Michael D. Bigham ""M. Wassir""",5,Great taffy,Great taffy at a great price. There was a wid...,0,0,30,2


In [ ]:
Amazon_Pred = Amazon[['Text','Text Length','Helpful', 'Not Helpful', 'Score']]
Amazon_Pred.head()

,Text,Text Length,Helpful,Not Helpful,Score
0,I have bought several of the Vitality canned d...,49,1,0,5
1,Product arrived labeled as Jumbo Salted Peanut...,31,0,0,1
2,This is a confection that has been around a fe...,99,1,0,4
3,If you are looking for the secret ingredient i...,43,3,0,2
4,Great taffy at a great price. There was a wid...,30,0,0,5


In [ ]:
Amazon_Pred.to_csv("Amazon_Pred.csv")

In [ ]:
Word_list = []
Word_dic = dict()
for i in list(Amazon_Pred['Text']):
    for j in i.split(' '):
        Word_list.append(j.upper())

In [ ]:
Word_list
import re

def clean_word(word):
    word = re.sub(r"(?i)(?<!\<)br(?!>)", "", word)
    word = re.sub(r"<.*?>|[^A-Za-z0-9\s]", "", word)
    word = word.strip()
    return word

Word_list = [clean_word(word) for word in Word_list if word and clean_word(word)]
Word_list

In [ ]:
def clean_text(text):
   text_arr = text.split(' ');
   text_arr = [clean_word(word) for word in text_arr if word and clean_word(word)]
   text = ' '.join(text_arr)
   return text

In [ ]:
len(Word_list)

45425052

In [ ]:
syc = ['THE','I','AND','A','TO','OF','IT','IS','THIS','FOR','IN','MY','THAT','BUT','WITH','HAVE','NOT','ARE','WAS','YOU','THEY',
       'AS','ON', 'LIKE','SO','THESE','BE','AT','OR','JUST','IF','THEM','ITs','WHEN','ALL','HAS','WE','WOULD','HAD',
       'MORE','THAN', 'BR']

Word_list = [word for word in Word_list if word not in syc]

len(Word_list)

27084735

In [ ]:
Word_dic = Counter(Word_list)
Word_df = pd.DataFrame({'Word':list(Word_dic.keys()),'Frequency':list(Word_dic.values())})
Word_df.sort_values('Frequency',ascending=False,inplace=True)
Word_df.head(20)

In [ ]:
Amazon_Prediction = Amazon[['Text', 'Summary','Helpful', 'Not Helpful', 'Score']]
Amazon_Prediction['Helpful_tier'] = Amazon['Helpful'] - Amazon['Not Helpful']
Amazon_Prediction.drop(['Helpful','Not Helpful'],axis=1,inplace=True)
Amazon_Prediction.head()

,Text,Summary,Score,Helpful_tier
0,I have bought several of the Vitality canned d...,Good Quality Dog Food,5,1
1,Product arrived labeled as Jumbo Salted Peanut...,Not as Advertised,1,0
2,This is a confection that has been around a fe...,"""Delight"" says it all",4,1
3,If you are looking for the secret ingredient i...,Cough Medicine,2,3
4,Great taffy at a great price. There was a wid...,Great taffy,5,0


In [ ]:
Amazon_Prediction.to_csv("Amazon_Prediction.csv")

In [ ]:
Amazon_Prediction['Text'] = Amazon_Prediction['Text'].apply(lambda x : clean_text(x))

In [ ]:
Amazon_Prediction['Summary'] = Amazon_Prediction['Summary'].apply(lambda x : clean_text(x))

In [ ]:
Amazon_Prediction['Reviews'] = Amazon_Prediction['Summary'] + Amazon_Prediction['Text']
Amazon_Prediction_R = Amazon_Prediction[['Reviews','Helpful_tier', 'Score']]
Amazon_Prediction_R.to_csv("Amazon_Prediction_R.csv")

In [ ]:
Amazon_Prediction_R

,Reviews,Helpful_tier,Score
0,Good Quality Dog FoodI have bought several of ...,1,5
1,Not as AdvertisedProduct arrived labeled as Ju...,0,1
2,Delight says it allThis is a confection that h...,1,4
3,Cough MedicineIf you are looking for the secre...,3,2
4,Great taffyGreat taffy at a great price There ...,0,5
...,...,...,...
568449,Will not do withoutGreat for sesame chickenthi...,0,5
568450,disappointedIm disappointed with the flavor Th...,0,2
568451,Perfect for our maltipooThese stars are small ...,2,5
568452,Favorite Training and reward treatThese are th...,1,5


In [ ]:
Amazon_Prediction_R.to_csv("Amazon_Prediction_R.csv")

In [5]:
data = pd.read_csv("Amazon_Prediction_R.csv")

In [ ]:
from sklearn.svm import SVR
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import mean_squared_error

# Split your dataset into input features (X) and target variable (y)
X = data['Reviews']
y = data['Helpful_tier']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Apply text vectorization to convert text data into numerical features
vectorizer = TfidfVectorizer()
X_train_vectorized = vectorizer.fit_transform(X_train)
X_test_vectorized = vectorizer.transform(X_test)

# Define the parameter grid for grid search
param_grid = {
    'C': [0.1, 1, 10],
    'epsilon': [0.1, 0.2, 0.5],
    'kernel': ['linear',  'poly', 'rbf', 'sigmoid', 'precomputed']
}

# Create the SVR model
model = SVR()

# Perform grid search cross-validation
grid_search = GridSearchCV(estimator=model, param_grid=param_grid, cv=5)
grid_search.fit(X_train_vectorized, y_train)

# Get the best parameters and best score
best_params = grid_search.best_params_
best_score = grid_search.best_score_

# Fit the SVR model with the best parameters
best_model = SVR(**best_params)
best_model.fit(X_train_vectorized, y_train)

# Predict with the best model
y_pred = best_model.predict(X_test_vectorized)

# Evaluate the model's performance using mean squared error (MSE)
mse = mean_squared_error(y_test, y_pred)
print("Mean Squared Error:", mse)
